In [53]:
# Print hello world
print("Hello world!")

# Setup the notebook
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


Hello world!


In [54]:
# Import the MNIST dataset and put it in a dataloader
train_loader = DataLoader(
    datasets.MNIST(
        'data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
)

In [55]:
# Import the test portion of the mnist dataset and put it in a dataloader
test_loader = DataLoader(
    datasets.MNIST(
        'data',
        train=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
)

In [56]:
# Define the training function
def train(model, loss_fn, train_loader, optimizer, epoch):
    # Use CUDA if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Set the model to training mode
    model.train()
    # Loop through the training data
    for batch_idx, (data, target) in enumerate(train_loader):
        # Send the data to the device
        data, target = data.to(device), target.to(device)
        # Zero the gradients
        optimizer.zero_grad()
        # Get the output of the model
        output = model(data)
        # Calculate the loss
        loss = F.cross_entropy(output, target)
        # Backpropagate the loss
        loss.backward()
        # Update the model parameters
        optimizer.step()
        # Print the loss every 100 batches
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                
            )

In [57]:
# Define the test function
def test(model, loss_fn, test_loader):
    # Use CUDA if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Set the model to evaluation
    model.eval()
    # Initialize the loss and accuracy
    test_loss = 0
    correct = 0
    # Loop through the test data
    with torch.no_grad():
        for data, target in test_loader:
            # Send the data to the device
            data, target = data.to(device), target.to(device)
            # Get the output of the model
            output = model(data)
            # Calculate the loss
            test_loss += loss_fn(output, target, reduction='sum').item()
            # Get the prediction
            pred = output.argmax(dim=1, keepdim=True)
            # Calculate the accuracy
            correct += pred.eq(target.view_as(pred)).sum().item()
            # Print the loss and accuracy
            test_loss /= len(test_loader.dataset)
            print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
                test_loss, correct, len(test_loader.dataset),
            ))

In [58]:
# Define the model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv22 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv32 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(576, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv12(x)
        x = F.relu(x)
        x = self.conv12(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=(2, 2))

        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv22(x)
        x = F.relu(x)
        x = self.conv22(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=(2, 2))

        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv32(x)
        x = F.relu(x)
        x = self.conv32(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=(2, 2))
        
        x = self.flatten(x)
        x = self.fc2(self.fc1(x))
        return F.log_softmax(x, dim=1)

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

In [59]:
#  Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the model
for epoch in range(1, 10):
    train(model, loss_fn, train_loader, optimizer, epoch)
    test(model, loss_fn, test_loader)

TypeError: CrossEntropyLoss.forward() got an unexpected keyword argument 'reduction'